In [6]:
import concurrent.futures

def extract_pdf_text(file_name):
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfparser import PDFParser
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfdevice import PDFDevice
    from pdfminer.pdfpage import PDFPage
    from pdfminer.layout import LAParams
    from pdfminer.layout import LTChar
    from pdfminer.converter import PDFPageAggregator
    
    print(file_name)
   
    # Open the PDF file
    fp = open(file_name, 'rb')

    # Create a PDF parser object associated with the file object
    parser = PDFParser(fp)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Create a PDF resource manager object that stores shared resources
    rsrcmgr = PDFResourceManager()

    # Set parameters for analysis
    laparams = LAParams()

    # Create a PDF page aggregator object
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Save the output of the code to a list
    output = []
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        layout = device.get_result()
        for obj in layout:
            if hasattr(obj, "get_text"):
                for line in obj:
                    for char in line:
                        if isinstance(char, LTChar):
                            output.append((char.get_text(), char.fontname))

    # Process the output word by word
    bold_word = ""
    italic_word = ""
    nested_list = []
    previous_font = ""

    for char, font in output:
        if "Arial-BoldItalicMT" in font:
            if "Arial-ItalicMT" in previous_font:
                nested_list.append([bold_word, italic_word])
                bold_word = ""
                italic_word = ""
            bold_word += char
            previous_font = font
        elif "Arial-ItalicMT" in font:
            italic_word += char
            previous_font = font
        else:
            if bold_word:
                nested_list.append([bold_word, italic_word])
                bold_word = ""
                italic_word = ""

    # Check if there's any remaining text that needs to be added to the list
    if bold_word:
        nested_list.append([bold_word, italic_word])

    # If the first element of a sublist is empty, but the second element isnt, add that to the second element of the previous sublist
    final_list = []
    for i in range(len(nested_list)):
        stripped_first_elem = nested_list[i][0].strip()
        if stripped_first_elem.startswith("ST") or stripped_first_elem.startswith("Ratspräsident") or stripped_first_elem.startswith('Pierre Heusser') or stripped_first_elem.startswith("Vizepräsident") or stripped_first_elem.startswith('Pierre Heusser') or any(char.isdigit() for char in stripped_first_elem) == True:
            continue
            
        elif nested_list[i][0].strip() == "" and nested_list[i][1].strip() != "":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
        
        elif nested_list[i][0].strip() == ":" and nested_list[i][1].strip() != "":
            if final_list:
                final_list[-1][1] += nested_list[i][1] 
                
        elif nested_list[i][0].strip() == ".":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
    
        elif nested_list[i][0].strip() == ".":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
        
        elif nested_list[i][0].strip() != "" and nested_list[i][0][:2] != "ST":

            final_list.append(nested_list[i])
        
    # Delete spaces at the beginning of the second element of every sublist
    for i in range(len(final_list)):
        final_list[i][1] = final_list[i][1].lstrip()
        
    # Delete "Dr." ...
    for i in range(len(final_list)):
        final_list[i][0] = final_list[i][0].replace('Dr.', '')    
        
    final_list = [sublist for sublist in final_list if len(sublist[0]) >= 10]
        

    print(final_list)
    return final_list

def process_files(files):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_pdf_text, file) for file in files]
        for future in concurrent.futures.as_completed(futures):
            print(future.result())

if __name__ == "__main__":
    files = ['GR-Protokoll 20180516.001 substanziell.pdf', 'GR-Protokoll 20180523.002 substanziell.pdf', 'GR-Protokoll 20180523.002 substanziell.pdf', 'GR-Protokoll 20180606.004 substanziell.pdf' ]
    process_files(files)


GR-Protokoll 20180516.001 substanziell.pdfGR-Protokoll 20180523.002 substanziell.pdf

GR-Protokoll 20180523.002 substanziell.pdf
GR-Protokoll 20180606.004 substanziell.pdf
[['Matthias Wiesmann (GLP):', 'Dass der Gemeinderat den Direktor der Finanzkontrolle – anders als die Beauftragte in Beschwerdesachen oder den Datenschutzbeauftragten – im Rahmen einer Weisung wählt, geht auf eine Volksabstimmung aus dem Jahr 2007 zurück. Damals hat man die Finanzkontrolle aus der Allgemeinen Verwaltung herausgelöst und dem Gemeinderat angegliedert. Dies wurde damit begründet, dass die Finanzkontrolle die Verwaltung kontrolliert und daher dieser nicht angehören kann. Die Wahl der Ombudsfrau und des Datenschutzbeauftragen lag hingegen schon immer beim Gemeinderat. Der Stadtrat hat bei der Wahl des Direktors der Finanzkontrolle jedoch weiterhin ein Vorschlagsrecht. Daher muss der Stadtrat dem Gemeinderat für diese Wahl auch eine Weisung vorlegen. Bei der gleichen Volksabstimmung 2007 hat man übrigens e